In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import DistilBertTokenizer,DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding


In [2]:
df = pd.read_csv("processed_reviews.csv")

# Check class distribution
print(df['sentiment'].value_counts())

sentiment
2    9022
1     772
0     467
Name: count, dtype: int64


In [3]:
x = list(df['reviews'])
y = list(df['sentiment'])

In [5]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    x,
    y,
    test_size=0.2,
    random_state=42,
    stratify=df['sentiment']
)

In [6]:
# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize datasets
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

In [8]:
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

In [9]:
# Load pre-trained BERT model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    save_strategy="epoch"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer)
)

# Train the model
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-9-3562514304.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: satkritikhadka2061 (satkritikhadka2061-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,0.740300
100,0.432900
150,0.417800
200,0.353900
250,0.356400
300,0.393800
350,0.328300
400,0.337900
450,0.355000
500,0.310400


TrainOutput(global_step=1539, training_loss=0.25549757333449374, metrics={'train_runtime': 360.0932, 'train_samples_per_second': 68.382, 'train_steps_per_second': 4.274, 'total_flos': 815483848863744.0, 'train_loss': 0.25549757333449374, 'epoch': 3.0})

In [10]:
predictions = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions),axis=1)
print("\nClassification")
print(classification_report(test_labels,preds.numpy(),target_names=["Negative","Neutral","Positive"]))


Classification
              precision    recall  f1-score   support

    Negative       0.66      0.55      0.60        93
     Neutral       0.45      0.43      0.44       155
    Positive       0.95      0.96      0.96      1805

    accuracy                           0.90      2053
   macro avg       0.69      0.65      0.67      2053
weighted avg       0.90      0.90      0.90      2053



In [12]:
model.eval()

text = 'This is a bad product.'
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

# Move inputs to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {key: val.to(device) for key, val in inputs.items()}
model.to(device)

outputs = model(**inputs)
logits = outputs.logits
predicted_class_id = torch.argmax(logits, dim=1).item()

print(predicted_class_id)

0
